In [1]:
import pandas as pd 
from sodapy import Socrata
import os
import csv

In [8]:
# url = https://data.austintexas.gov/resource/7d8e-dm7r.json
# Data Extraction:
client = Socrata("data.austintexas.gov", None)

results = client.get("7d8e-dm7r")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [9]:
clean_results_df = results_df.copy()

In [10]:
clean_results_df = clean_results_df.rename(columns = {
    "trip_id": "Trip ID",
    "device_id": "Device ID",
    "modified_date": "Data Modified Date",
    "vehicle_type": "Vehicle Type",
    "trip_duration": "Trip Duration",
    "trip_distance": "Trip Distance",
    "start_time": "Trip Start Time",
    "end_time": "Trip End Time",
    "hour": "Hour",
    "day_of_week": "Day Of Week",
    "month": "Month",
    "year": "Year",
    "census_geoid_start": "GEOID Start",
    "census_geoid_end": "GEOID End",
    "council_district_start": "Start Council District",
    "council_district_end" : "Return Council District"
})

In [11]:
# Change the time and date format for columns - 'Data Modified Date', 'Trip Start time' and 'Trip End Time'
clean_results_df['Trip Start Time'] = pd.to_datetime(clean_results_df['Trip Start Time'])
clean_results_df['Trip End Time'] = pd.to_datetime(clean_results_df['Trip End Time'])
clean_results_df['Data Modified Date'] = pd.to_datetime(clean_results_df['Data Modified Date'])
clean_results_df.head()

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,2018-12-13 15:36:12,0,0,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,2018-12-21 00:41:42,9,0,bicycle,11491603,1658,2018-11-30 11:00:00,2019-04-12 12:00:00,11,11,5,2018,NaN,NaN
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30 06:45:17,9,9,scooter,170,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30 06:45:17,9,9,scooter,90,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,2019-04-30 06:45:17,9,9,scooter,570,826,2019-04-29 17:15:00,2019-04-29 17:15:00,4,17,1,2019,48453001100,48453001100


In [12]:
csvpath = os.path.join("ZIP_TRACT.csv")


zip_data = pd.read_csv(csvpath)
zip_data

             

zip_data[["zip", "GEOID End"]] = zip_data[["zip", "GEOID End"]].astype(str)

zip_data


zip_data_merge = pd.merge(clean_results_df,zip_data, on= "GEOID End", how ="right" )

zip_data_merge.dropna()

merged_df = zip_data_merge.drop(columns=['res_ratio', 'bus_ratio', 'oth_ratio','tot_ratio']).dropna()

In [13]:
merged_df

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,Month,Hour,Day Of Week,Year,GEOID Start,GEOID End,zip
0,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30 06:45:17,9,9,scooter,170,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601,78705
1,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30 06:45:17,9,9,scooter,90,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601,78705
2,63bbe554-d468-4c76-9101-eb8a6731525c,c3d2f2c5-3762-41f4-b5e6-71d5b714f731,2019-04-30 06:45:17,9,9,scooter,477,1144,2019-04-29 17:45:00,2019-04-29 17:45:00,4,17,1,2019,48453000601,48453000601,78705
3,d705a126-24bd-4054-bda0-817e58f89e74,5aaf53be-efc0-46ec-84c6-34b832d1af29,2019-04-30 06:45:17,9,9,scooter,49,52,2019-04-29 17:15:00,2019-04-29 17:15:00,4,17,1,2019,48453000601,48453000601,78705
4,e4454d26-808b-4d4e-a629-ba48f9f7cdfb,ea543ab8-c3d6-4fc3-a0ae-f298cf493340,2019-04-30 06:45:17,1,9,scooter,125,444,2019-04-29 17:15:00,2019-04-29 17:15:00,4,17,1,2019,48453000401,48453000601,78705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747,9c8f4790-f79b-4d42-955a-267c45103151,10320afe-1428-4d04-b85d-a5e0033b3447,2019-04-30 07:00:30,3,3,scooter,1572,4940,2019-04-29 18:15:00,2019-04-29 18:45:00,4,18,1,2019,48453002307,48453002003,78704
2748,9c8f4790-f79b-4d42-955a-267c45103151,10320afe-1428-4d04-b85d-a5e0033b3447,2019-04-30 07:00:30,3,3,scooter,1572,4940,2019-04-29 18:15:00,2019-04-29 18:45:00,4,18,1,2019,48453002307,48453002003,78745
2749,b275f528-e606-4f96-88d3-df81af542ddf,4752f34a-9ea1-4d12-b130-a2f64f41969b,2019-04-30 07:00:30,3,3,scooter,620,2204,2019-04-29 18:15:00,2019-04-29 18:30:00,4,18,1,2019,48453000801,48453000801,78702
2750,e0d26f49-f7fc-44aa-9e75-135d74e61acb,44ee0fab-adac-432d-906a-3b9bac75a387,2019-04-30 07:00:30,3,3,scooter,253,450,2019-04-29 18:00:00,2019-04-29 18:00:00,4,18,1,2019,48453000902,48453000801,78702


In [14]:
client = Socrata("data.austintexas.gov", None)
results = client.get("i26j-ai4z", limit=900000)
results_df = pd.DataFrame.from_records(results)


#results_df.sort_values("sr_type_code",ascending = False )

results_df.sr_type_code.unique() 

shared_mobility = results_df.loc[results_df['sr_type_desc'] == "Shared Micromobility"]



shared_mobility 

,sr_number,sr_type_code,sr_type_desc,sr_department_desc,sr_method_received_desc,sr_status_desc,sr_status_date,sr_created_date,sr_updated_date,sr_location,...,sr_location_map_page,sr_location_map_tile,:@computed_region_8spj_utxs,:@computed_region_jcrc_4uuy,:@computed_region_q9nd_rr82,:@computed_region_e9j2_6w3z,:@computed_region_m2th_e4b7,:@computed_region_rxpj_nzrk,:@computed_region_a3it_2a2z,sr_closed_date
11,20-00013542,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-11T20:57:03.000,2020-01-11T20:57:03.000,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",...,585C,MK24,9,43,10,50,318,50,2860,NaN
12,20-00013540,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-11T20:55:50.000,2020-01-11T20:55:50.000,2020-01-11T20:55:50.000,"3300 N IH 35 SVRD SB, AUSTIN, TX 78705",...,585C,MK24,9,43,10,50,318,50,2860,NaN
16,20-00013532,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-11T20:43:36.000,2020-01-11T20:43:36.000,2020-01-11T20:43:36.000,"1505 E 14TH ST, AUSTIN, TX 78702",...,585Q,MK23,1,7,8,56,309,1,2857,NaN
24,20-00013518,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-11T20:22:37.000,2020-01-11T20:22:37.000,2020-01-11T20:22:37.000,"315 CONGRESS AVE, AUSTIN, TX 78701",...,585W,MJ22,9,52,10,60,217,24,2856,NaN
34,20-00013477,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-11T18:58:56.000,2020-01-11T18:58:56.000,2020-01-11T18:58:56.000,"322 CONGRESS AVE, AUSTIN, TX 78701",...,585W,MJ22,9,52,10,60,217,24,2856,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80583,19-00281614,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-08-02T13:09:49.000,2019-07-24T20:23:08.000,2019-08-02T13:09:49.000,"115 SANDRA MURAIDA WAY, AUSTIN, TX 78703",...,584Z,MH22,9,52,10,60,217,69,2856,2019-08-02T13:09:49.000
80634,19-00281506,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:45:28.000,2019-07-24T18:55:41.000,2019-07-31T09:45:28.000,"82 RAINEY ST, AUSTIN, TX 78701",...,615B,MJ21,9,52,10,60,502,24,2856,2019-07-31T09:45:28.000
80955,19-00280629,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:08:40.000,2019-07-24T10:18:53.000,2019-07-31T09:08:40.000,"404 W 26TH ST, AUSTIN, TX 78705",...,585E,MJ24,9,43,10,44,217,50,2860,2019-07-31T09:08:40.000
80994,19-00280569,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:10:05.000,2019-07-24T09:57:56.000,2019-07-31T09:10:05.000,"514 W 24TH ST, AUSTIN, TX 78705",...,585E,MJ24,9,43,10,44,217,50,2860,2019-07-31T09:10:05.000


In [15]:
scooter_df = pd.DataFrame({
    "complaint_number": shared_mobility["sr_number"],
    "complaint_type": shared_mobility["sr_type_desc"],
    "complaint_date": shared_mobility["sr_created_date"],
    "complaint_location": shared_mobility["sr_location"],
    "complaint_zip": shared_mobility["sr_location_zip_code"],
    "complaint_latitude": shared_mobility["sr_location_lat"],
    "complaint_longitudge": shared_mobility["sr_location_long"]
                                            })

scooter_df

,complaint_number,complaint_type,complaint_date,complaint_location,complaint_zip,complaint_latitude,complaint_longitudge
11,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",78705,30.29341349,-97.72199013
12,20-00013540,Shared Micromobility,2020-01-11T20:55:50.000,"3300 N IH 35 SVRD SB, AUSTIN, TX 78705",78705,30.29130734,-97.72273781
16,20-00013532,Shared Micromobility,2020-01-11T20:43:36.000,"1505 E 14TH ST, AUSTIN, TX 78702",78702,30.27515024,-97.72397411
24,20-00013518,Shared Micromobility,2020-01-11T20:22:37.000,"315 CONGRESS AVE, AUSTIN, TX 78701",78701,30.26563119,-97.74337971
34,20-00013477,Shared Micromobility,2020-01-11T18:58:56.000,"322 CONGRESS AVE, AUSTIN, TX 78701",78701,30.26615222,-97.74398437
...,...,...,...,...,...,...,...
80583,19-00281614,Shared Micromobility,2019-07-24T20:23:08.000,"115 SANDRA MURAIDA WAY, AUSTIN, TX 78703",78703,30.26755559,-97.75494956
80634,19-00281506,Shared Micromobility,2019-07-24T18:55:41.000,"82 RAINEY ST, AUSTIN, TX 78701",78701,30.25964236,-97.7385377
80955,19-00280629,Shared Micromobility,2019-07-24T10:18:53.000,"404 W 26TH ST, AUSTIN, TX 78705",78705,30.290531,-97.74153837
80994,19-00280569,Shared Micromobility,2019-07-24T09:57:56.000,"514 W 24TH ST, AUSTIN, TX 78705",78705,30.28795653,-97.74302556


In [16]:
#scooter_df.complaint_zip.unique() 


scooter = scooter_df["complaint_zip"].value_counts()

scooter


78701    1347
78704     858
78705     308
78702     296
78703     250
78746     234
78741      73
78751      49
78723      31
78756      29
78712      22
78731      13
78745      13
78722      11
78664      11
78754      10
78757      10
78752      10
78744       9
78721       7
78758       7
78747       6
78748       5
78753       4
78681       4
78717       3
78759       2
78725       2
78736       1
78613       1
78727       1
78617       1
78660       1
78750       1
78749       1
Name: complaint_zip, dtype: int64

,zip,GEOID End,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,501,36103158607,0.000000,1.000000,0.000000,1.000000
1,601,72113071700,0.188722,0.209068,0.125000,0.188301
2,601,72001956800,0.017797,0.025189,0.083333,0.020032
3,601,72001956300,0.004934,0.005038,0.005952,0.004968
4,601,72001956600,0.166520,0.347607,0.345238,0.182853
...,...,...,...,...,...,...
170188,99925,2198000200,0.000000,0.000000,1.000000,1.000000
170189,99926,2198940100,0.000000,0.000000,1.000000,1.000000
170190,99927,2198000100,0.000000,0.000000,1.000000,1.000000
170191,99928,2130000100,0.000000,0.000000,1.000000,1.000000


In [17]:
scooter_df = pd.DataFrame({
    "Complaint Number": shared_mobility["sr_number"],
    "Complaint Type": shared_mobility["sr_type_desc"],
    "Complaint Date": shared_mobility["sr_created_date"],
    "complaint_location": shared_mobility["sr_location"],
    "zip": shared_mobility["sr_location_zip_code"],
    "complaint_latitude": shared_mobility["sr_location_lat"],
    "complaint_longitudge": shared_mobility["sr_location_long"]
                                           })

scooter_df


,Complaint Number,Complaint Type,Complaint Date,complaint_location,zip,complaint_latitude,complaint_longitudge
11,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",78705,30.29341349,-97.72199013
12,20-00013540,Shared Micromobility,2020-01-11T20:55:50.000,"3300 N IH 35 SVRD SB, AUSTIN, TX 78705",78705,30.29130734,-97.72273781
16,20-00013532,Shared Micromobility,2020-01-11T20:43:36.000,"1505 E 14TH ST, AUSTIN, TX 78702",78702,30.27515024,-97.72397411
24,20-00013518,Shared Micromobility,2020-01-11T20:22:37.000,"315 CONGRESS AVE, AUSTIN, TX 78701",78701,30.26563119,-97.74337971
34,20-00013477,Shared Micromobility,2020-01-11T18:58:56.000,"322 CONGRESS AVE, AUSTIN, TX 78701",78701,30.26615222,-97.74398437
...,...,...,...,...,...,...,...
80583,19-00281614,Shared Micromobility,2019-07-24T20:23:08.000,"115 SANDRA MURAIDA WAY, AUSTIN, TX 78703",78703,30.26755559,-97.75494956
80634,19-00281506,Shared Micromobility,2019-07-24T18:55:41.000,"82 RAINEY ST, AUSTIN, TX 78701",78701,30.25964236,-97.7385377
80955,19-00280629,Shared Micromobility,2019-07-24T10:18:53.000,"404 W 26TH ST, AUSTIN, TX 78705",78705,30.290531,-97.74153837
80994,19-00280569,Shared Micromobility,2019-07-24T09:57:56.000,"514 W 24TH ST, AUSTIN, TX 78705",78705,30.28795653,-97.74302556


In [20]:
combined_df = pd.merge(merged_df,scooter_df, on= "zip", how ="right" )

clean_combined = combined_df.dropna()

clean_combined

,Trip ID,Device ID,Data Modified Date,Start Council District,Return Council District,Vehicle Type,Trip Duration,Trip Distance,Trip Start Time,Trip End Time,...,Year,GEOID Start,GEOID End,zip,Complaint Number,Complaint Type,Complaint Date,complaint_location,complaint_latitude,complaint_longitudge
0,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30 06:45:17,9,9,scooter,170,0,2019-04-29 17:30:00,2019-04-29 17:30:00,...,2019,48453000601,48453000601,78705,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",30.29341349,-97.72199013
1,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30 06:45:17,9,9,scooter,90,0,2019-04-29 17:30:00,2019-04-29 17:30:00,...,2019,48453000601,48453000601,78705,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",30.29341349,-97.72199013
2,63bbe554-d468-4c76-9101-eb8a6731525c,c3d2f2c5-3762-41f4-b5e6-71d5b714f731,2019-04-30 06:45:17,9,9,scooter,477,1144,2019-04-29 17:45:00,2019-04-29 17:45:00,...,2019,48453000601,48453000601,78705,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",30.29341349,-97.72199013
3,d705a126-24bd-4054-bda0-817e58f89e74,5aaf53be-efc0-46ec-84c6-34b832d1af29,2019-04-30 06:45:17,9,9,scooter,49,52,2019-04-29 17:15:00,2019-04-29 17:15:00,...,2019,48453000601,48453000601,78705,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",30.29341349,-97.72199013
4,e4454d26-808b-4d4e-a629-ba48f9f7cdfb,ea543ab8-c3d6-4fc3-a0ae-f298cf493340,2019-04-30 06:45:17,1,9,scooter,125,444,2019-04-29 17:15:00,2019-04-29 17:15:00,...,2019,48453000401,48453000601,78705,20-00013542,Shared Micromobility,2020-01-11T20:57:03.000,"3500 HARMON AVE, AUSTIN, TX 78705",30.29341349,-97.72199013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042002,7e405474-7636-4f31-aaac-0d57bcac2c98,91136bf9-be08-4104-a66d-1d74b9db386e,2019-04-30 06:45:17,10,10,scooter,139,115,2019-04-29 17:15:00,2019-04-29 17:15:00,...,2019,48453000204,48453000101,78731,19-00296255,Shared Micromobility,2019-08-04T20:04:44.000,"2801 W FRESCO DR, AUSTIN, TX 78731",30.32658726,-97.74980839
1042003,1aa6a762-245e-403c-b381-d5ffec2a95e4,839bcefb-7740-49e7-943c-d33453d27b25,2019-04-30 07:00:30,9,10,scooter,670,2478,2019-04-29 18:15:00,2019-04-29 18:30:00,...,2019,48453000204,48453000101,78731,19-00296255,Shared Micromobility,2019-08-04T20:04:44.000,"2801 W FRESCO DR, AUSTIN, TX 78731",30.32658726,-97.74980839
1042004,6b17e26a-a7fb-4376-975e-1067443da224,cd40fcab-5055-4840-bc49-06e22883a993,2019-04-30 06:45:17,9,10,scooter,1291,6069,2019-04-29 17:15:00,2019-04-29 17:30:00,...,2019,48453000603,48453000101,78731,19-00294793,Shared Micromobility,2019-08-03T10:10:23.000,"2720 HANCOCK DR, AUSTIN, TX 78731",30.32770095,-97.74883131
1042005,7e405474-7636-4f31-aaac-0d57bcac2c98,91136bf9-be08-4104-a66d-1d74b9db386e,2019-04-30 06:45:17,10,10,scooter,139,115,2019-04-29 17:15:00,2019-04-29 17:15:00,...,2019,48453000204,48453000101,78731,19-00294793,Shared Micromobility,2019-08-03T10:10:23.000,"2720 HANCOCK DR, AUSTIN, TX 78731",30.32770095,-97.74883131
